_See [Readme](https://github.com/fleuryc/oc_ingenieur-ia_P2-Participez-a-un-concours-sur-la-Smart-City#readme) for installation instructions_

---


# Concours Data is for Good : aidons Paris à devenir une smart-city !

## Contexte

Dans le cadre du programme "Végétalisons la ville" organisé par la ville de Paris, nous proposons ici une analyse exploratoire des données OpenData concernant les arbres gérés par la ville de Paris.

L'objectif est d'aider Paris à devenir une "Smart-City" en gérant ses arbres de la manière la plus responsable possible. C'est-à-dire en optimisant les trajets nécessaires pour entretenir ces arbres.


## Outils utilisés

Nous allons utiliser le langage Python, et présenter ici le code, les résultats et l'analyse sous forme de [Notebook Jupyter](https://jupyterlab.readthedocs.io/en/stable/getting_started/overview.html).

Nous allons aussi utiliser les bibliothèques usuelles d'exploration et analyse de données, afin d'améliorer la simplicité et la performance de notre code :
* [NumPy](https://numpy.org/doc/stable/user/quickstart.html) et [Pandas](https://pandas.pydata.org/docs/user_guide/index.html) : effectuer des calculs scientifiques (statistiques, algèbre, ...) et manipuler des séries et tableaux de données volumineuses et complexes
* [Matplotlib](https://matplotlib.org/stable/tutorials/introductory/usage.html), [Pyplot](https://matplotlib.org/stable/tutorials/introductory/pyplot.html), [Seaborn](https://seaborn.pydata.org/tutorial/function_overview.html) et [Plotly](https://plotly.com/python/getting-started/) : générer des graphiques lisibles, intéractifs et pertinents


In [27]:
# Import libraries
from zipfile import ZipFile
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## If you use Notebook (and not JupyterLab), uncomment following lines
# import plotly.io as pio
# pio.renderers.default='notebook'


## Chargement des données et premier aperçu

Les données mises à disposition sont issues de  [opendata.paris.fr](https://opendata.paris.fr/explore/dataset/les-arbres/information/) et représentent "l’ensemble des arbres, ainsi que les arbres d’alignement, présents sur le territoire parisien et des cimetières extra-muros (hors de Paris)."



Nous allons dans un premier temps simplement charger les données en mémoire et observer quelques valeurs.

In [33]:

csv_filename = 'fr.openfoodfacts.org.products.csv'
zip_filename = csv_filename+'.zip'
zip_url = 'https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/'+zip_filename

data_local_path = 'data/'
zip_local_path = data_local_path+zip_filename
csv_local_path = data_local_path+csv_filename

r = requests.get(zip_url)
with open(zip_local_path, 'wb') as f:
    f.write(r.content)

with ZipFile(zip_local_path, 'r') as zip_file:
    zip_file.extractall(data_local_path)


In [42]:

raw_data = pd.read_csv(csv_local_path, sep='\t', dtype={
    'code': str,
    'created_t': 'datetime64[ns]',
})


TypeError: the dtype datetime64[ns] is not supported for parsing, pass this column using parse_dates instead

In [37]:
# display first 5 rows
raw_data.head()


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,NaN,NaN
2,0000000004559,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Peanuts,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
3,0000000016087,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055731,2017-03-09T10:35:31Z,1489055731,2017-03-09T10:35:31Z,Organic Salted Nut Mix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN
4,0000000016094,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055653,2017-03-09T10:34:13Z,1489055653,2017-03-09T10:34:13Z,Organic Polenta,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:

# Display data types and empty values
raw_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Columns: 162 entries, code to water-hardness_100g
dtypes: float64(106), object(56)
memory usage: 396.5+ MB


In [39]:

# Display statistical summary of each column
raw_data.describe(include="all")


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
count,320749,320749,320770,3.207690e+05,320763,3.207720e+05,320772,303010,52795,104819,...,49.000000,3036.000000,165.000000,948.000000,0.0,268.000000,221210.000000,221210.000000,0.0,0.0
unique,320749,320749,3535,1.896360e+05,189568,1.806390e+05,180495,221347,38584,13826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,0037600137614,http://world-fr.openfoodfacts.org/produit/3760...,usda-ndb-import,1.489056e+09,2017-03-09T10:37:09Z,1.439142e+09,2015-08-09T17:35:42Z,Ice Cream,Pâtes alimentaires au blé dur de qualité supér...,500 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1,169868,2.000000e+01,20,3.300000e+01,33,410,201,4669,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.425698,31.458587,15.412121,49.547785,NaN,341.700764,9.165535,9.058049,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.047841,31.967918,3.753028,18.757932,NaN,425.211439,9.055903,9.183589,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,8.000000,6.000000,NaN,0.000000,-15.000000,-15.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.300000,0.000000,12.000000,32.000000,NaN,98.750000,1.000000,1.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.200000,23.000000,15.000000,50.000000,NaN,195.750000,10.000000,9.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.400000,51.000000,15.000000,64.250000,NaN,383.200000,16.000000,16.000000,NaN,NaN


Nous voyons que, pour chaque arbre listé, nous disposons des informations suivantes (la description des colonnes est disponible sur le site [OpenData](https://opendata.paris.fr/explore/dataset/les-arbres/information/)) :
- `id` : simple identifiant de l'arbre (entier, ex. : `99874`)
- `type_emplacement` : type de l'emplacement (texte, ex. : `"Arbre"`)
- `domanialite` : type de lieu auquel appartient l'arbre (texte, ex. : `"Jardin"`)
- `arrondissement` : arrondissement de Paris où est situé l'arbre (texte, ex. : `"PARIS 7E ARRDT"`)
- `complement_addresse` : complement d'adress (texte, pas d'exemple visible)
- `numero` : numéro de l'adress (texte, pas d'exemple visible)
- `lieu` : adresse de l'arbre (texte, ex. : `"MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E"`)
- `id_emplacement` : identifiant de l'emplacement (texte, ex. : `"19"`)
- `libelle_francais` : nom commun (vernaculaire) de l'espèce de l'arbre (texte, ex. : `"Marronnier"`)
- `genre` : genre de l'arbre (texte, ex. : `"Aesculus"`)
- `espece` : espèce de l'arbre (texte, ex. : `"hippocastanum"`)
- `variete` : variété de l'arbre (texte, pas d'exemple visible)
- `circonference_cm` : circonférence en centimètres de l'arbre (entier, ex. : `20`)
- `hauteur_m` : taille en mètres de l'arbre (entier, ex. : `5`)
- `stade_developpement` : stade de développement de l'arbre (texte, ex. : `"A"` pour "Adulte")
- `remarquable` : si l'arbre est "remarquable" ou non (booléen, ex. : `0` pour un arbre "non remarquable")
- `geo_point_2d_a` : latitude de la position de l'arbre (nombre à virgule, ex. : `48.857620`)
- `geo_point_2d_b` : longitude de la position de l'arbre (nombre à virgule, ex. : `2.320962`)

Nous voyons déjà que parmis les quelques premières données :
- un certain certain nombre de valeurs ne sont pas fournies (`NaN` = "Not a Number" = donnée non disponible)
- nous pouvons classer les variables selon leur type :
    - quantitatives
        - discrètes : `id`, `circonference_cm`, `hauteur_m`
        - continues : `geo_point_2d_a`, `geo_point_2d_b`
    - qualitatives
        - nominales : `type_emplacement`, `domanialite`, `arrondissement`, `complement_addresse`, `numero`, `lieu`, `id_emplacement`, `libelle_francais`, `genre`, `espece`, `variete`
        - ordinales : `stade_developpement`, `remarquable`
- on peut aussi les classer en trois grandes catégories, d'après leur sens :
    - métadonnées internes au système : `id`, `id_emplacement`, `type_emplacement`
    - données de localisation : `arrondissement`, `complement_addresse`, `numero`, `lieu`, `geo_point_2d_a`, `geo_point_2d_b`
    - données de description : 
        - taille : `circonference_cm`, `hauteur_m` et `stade_developpement`
        - type : `libelle_francais`, `genre`, `espece` et `variete`
        - autre : `remarquable`


Nous allons observer plus précisément les types de valeurs et les valeurs vides :

In [ ]:
# Display data types and empty values
raw_data.info()


Nous voyons alors que :
- la colonne `numero` n'est jamais renseignée (`Non-Null count = 0`)
    - ce critère n'apporte donc pas d'information
- les colonnes `complement_addresse` (`Non-Null count = 30902`) et `variete` (`Non-Null count = 36777`) sont très peu renseignées (> 80% de valeurs non définies)
    - les informations apportées par ces colonnes seront donc très difficilement exploitables en l'état
- les colonnes `stade_developpement` (`Non-Null count = 132932`) et `remarquable` (`Non-Null count = 137039`) sont partiellement renseignées (> 30% de valeurs non définies)
    - les informations apportées par ces colonnes seront donc pas facilement exploitables en l'état
- les colonnes `libelle_francais` (`Non-Null count = 198640`) et `espece` (`Non-Null count = 198385`) sont pas toujours renseignées (> 0,5% de valeurs non définies)
    - les informations apportées par ces colonnes sont assez fiables, mais il faudra faire attention aux cas non renseignés


## Première analyse statistique

Nous allons maintenant chercher à comprendre comment sont réparties les valeurs pour chaque caractéristique de nos arbres.



Une simple description statistique de chaque colonne nous donne les informations suivantes :
- pour chaque donnée numérique (`id`, `circonference_cm`, `hauteur_m`, `remarquable`, `geo_point_2d_a` et `geo_point_2d_b`), nous obtenons :
    - le nombre de valeurs non vides (`count`)
    - la moyenne (`mean`)
    - l'écart-type (`std`)
    - les valeurs minimale (`min`) et maximale (`max`)
    - les 25, 50 (médiane) et 75 centiles (`25%`, `50%` et `75%`)

- our chaque donnée textuelle (`type_emplacement`, `domanialite`, `arrondissement`, `complement_addresse`, `lieu`, `id_emplacement`, `libelle_francais`, `genre`, `espece`, `variete` et `stade_developpement`), nous obtenons :
    - le nombre de valeurs non vides (`count`)
    - le nombre de valeurs différentes (`unique`)
    - la valeur la plus représentée (`top`)
    - la fréquence de la valeur la plus représentée (`freq`)


In [ ]:
# Display statistical summary of each column
raw_data.describe(include="all")


Observons maintenant la distribution empirique de chaque variable, de manière non visuelle dans un premier temps, afin de voir quels types de graphes seront ensuite le plus adaptés :

In [ ]:
# display value frequencies per column
for col in raw_data.columns:
    print(f'\n \
================================================\n \
>    { col }\n \
------------------------------------------------')

    counts = raw_data[col].value_counts()
    freq = raw_data[col].value_counts(normalize=True)
    display(pd.DataFrame({'count': counts, 'freq': freq}))
    


Nous voyons alors que :
- chaque arbre possède un `id` unique
    - cette variable n'apporte donc aucun information
- il n'y a qu'une seule valeur possible pour la variable `type_emplacement` : `"Arbre"`
    - ce critère n'apporte donc pas d'information
- les valeurs respectives de `complement_addresse` et `id_emplacement` sont très disparates dans leur format (pas de valeurs très représentatives) et ne sont pas humainement parlantes
- la colonne `lieu` peut être découpée avec le séparateur `" / "` afin de regrouper par exemple tous les lieux commenant par `"CIMETIERE DE PANTIN"`
- les données de `circonference_cm` et `hauteur_m` ont des valeurs aberrantes dont il faudra tenir compte :
    - `minimum = 0` , ce qui semble impossible
    - `circonference_cm` : `maximum = 250255` et `hauteur_m` : `maximum = 881818` , ce qui semble impossible


## Un peu de nettoyage

Nous allons :
- supprimer les colonnes inutiles : `type_emplacement` et `numero`
- renommer les valeurs de `stade_developpement` pour des valeurs plus explicites
- découper la valeur de `lieu` avec le séparateur `" / "`
- créer de nouvelles colonnes `top_XXX` où les valeurs les moins fréquentes seront remplacées par la valeur `"Other"` pour les colonnes `lieu`, `lieu_1`, `libelle_francais`, `genre`, `espece` et `variete`

In [ ]:
# drop useless columns
clean_data = raw_data.drop(columns=['type_emplacement','numero'])

# replace `stade_developpement` values
clean_data.stade_developpement.replace({
    'J' :'Jeune', 
    'JA':'Jeune Adulte', 
    'A' :'Adulte', 
    'M' :'Mature',
}, inplace=True)

# extract the first part of column `lieu`
clean_data['lieu_1'] = clean_data["lieu"].str.split("/", expand=True)[0].str.strip()


In [ ]:
# Display top 10 values of lieu and lieu_1
fig, (ax1, ax2) = plt.subplots(2, 1, 
    figsize=(16,12),
)

clean_data['lieu'].value_counts().head(10).plot(
    kind='barh', 
    ax=ax1,
    title="Top 10 lieux sans découpage ' / '",
)

clean_data['lieu_1'].value_counts().head(10).plot(
    kind='barh', 
    ax=ax2,
    title="Top 10 lieux avec découpage ' / '",
)

plt.show()


In [ ]:
# Let's keep only the top values and merge the rest into "Other"
for col in ['lieu', 'lieu_1', 'libelle_francais', 'genre', 'espece', 'variete']:
    freq = clean_data[col].value_counts()
    clean_data['top_'+col] = clean_data[col].where(
        clean_data[col].isna() | clean_data[col].isin(freq.index[:20]), 
        other='Other', 
    )


In [ ]:
# Let's see if categories are well organised
fig = px.parallel_categories(clean_data,
    dimensions=['arrondissement', 'top_lieu_1', 'top_lieu'],
    title="Classification des lieux",
    width=1000,
    height=800,
)
fig.show()

fig = px.parallel_categories(clean_data,
    dimensions=['top_libelle_francais', 'top_genre', 'top_espece', 'top_variete'],
    title="Classification des variétés d'arbres",
    width=1000,
    height=800,
)
fig.show()


## Améliorons la gestion des arbres

Nous allons ici nous appuyer sur des analyses statistiques et des graphiques afin de voir comment il serait possible d'améliorer le service de gestion des arbres de Paris.



### Quels arbres faut-il mesurer à nouveau ?

Pour la suite de l'analyse, nous allons éliminer les données abberrantes ("outliers). Pour celà, nous allons utiliser le critère [IQR](https://en.wikipedia.org/wiki/Interquartile_range#Outliers). Nous allons considérer toutes les données de taille trop éloignées de la norme, ainsi que les valeurs égales à `0` comme des données aberrantes.

Nous allons dans un premier temps afficher une cartographie de ces arbres, car ceux-ci devront être mesurés à nouveau afin d'améliorer la fiabilité de la gestion de nos arbres.
Nous allons ensuite considérer ces données comme nulles (`NaN`).


In [ ]:
# Let's work on a copy of our clean data.
data = clean_data.copy()

# First, let's consider zeros as NaN
data['circonference_cm'].where(data['circonference_cm'] > 0, inplace=True)
data['hauteur_m'].where(data['hauteur_m'] > 0, inplace=True)

# Let's compute the InterQuartile range in order to identify outliers
quartiles = data[['circonference_cm', 'hauteur_m']].quantile([0.25, 0.75])
iqr = quartiles.loc[0.75]-quartiles.loc[0.25]
limits = pd.DataFrame({
    'circonference_cm': [
        max(0, quartiles.loc[0.25,'circonference_cm'] - 1.5 * iqr['circonference_cm']), # min
        quartiles.loc[0.75,'circonference_cm'] + 1.5 * iqr['circonference_cm'], # max
    ],
    'hauteur_m': [
        max(0, quartiles.loc[0.25,'hauteur_m'] - 1.5 * iqr['hauteur_m']), # min
        quartiles.loc[0.75,'hauteur_m'] + 1.5 * iqr['hauteur_m'], # max
    ]
}, index=['min', 'max'])

display(quartiles, limits)



Nous voyons qu'un arbre "normal" aura :
- une circonférence comprise entre 0 et 240 cm
- une hauteur comprise entre 0 et 26 m

Nous allons maintenant visualiser où sont situé ces arbres "anormaux" (outliers), afin de planifier les tournées de mesure de ces arbres.


In [ ]:
# outliers are the trees outside the IQR range
outliers = clean_data[
    ( clean_data['circonference_cm'] <= limits.loc['min','circonference_cm'] )
    | ( clean_data['circonference_cm'] >= limits.loc['max','circonference_cm'] )
    | ( clean_data['hauteur_m'] <= limits.loc['min','hauteur_m'] )
    | ( clean_data['hauteur_m'] >= limits.loc['max','hauteur_m'] )
]


In [ ]:
# Count trees per burrough
count_per_arrondissement = outliers['arrondissement'].value_counts().head(20)

# resize figure
plt.figure(figsize=(16,9))

# plot horizontal bar chart
plt.barh(
    y=count_per_arrondissement.index,
    width=count_per_arrondissement.values,
)

# add labels for the value of each bar
for index, value in enumerate(count_per_arrondissement):
    plt.text(y=index , x=value+1 , s=f"{value}")

# add title and labels
plt.xlabel("Nombre d'arbres")
plt.ylabel("Arrondissement")
plt.title(f"Nombre d'arbres à re-mesurer par arrondissement.\nTOTAL = { len(outliers) }")

# display the figure
plt.show()


In [ ]:
# Display the outliers on a map
fig = px.density_mapbox(outliers, 
    lat='geo_point_2d_a', lon='geo_point_2d_b',
    hover_data=['circonference_cm', 'hauteur_m', 'arrondissement', 'lieu', 'domanialite'],
    radius=2,
    zoom=10,
    mapbox_style="open-street-map",
    title="Localisation des arbres à re-mesurer",
    width=1000,
    height=800,
)
fig.show()



Nous voyons ici la carte des 44171 arbres qu'il faudrait mesurer à nouveau.

En attendant que ces arbres soient mesurés à nouveau, nous allons maintenant les ignorer dans nos prochaines analyses : passer à `NaN` les valeurs aberrantes.


In [ ]:
# set to NaN data that are outside the range
for col in ['circonference_cm', 'hauteur_m']:
    clean_data[col] = clean_data[col].where(( 
        ( limits.loc['min', col] < clean_data[col] )
        & ( clean_data[col] < limits.loc['max', col] ) 
    ))


### Quels arbres ont un développement anormal ?

Afin de gérer efficacement le patrimoine arboricole, il faut être capable de détecter les potentiels arbres malades ou qui ont des problèmes de développement.

Nous allons ici chercher quels abres semblent avoir un développement anormal et donc qu'il faudrait contrôler en priorité.


In [ ]:
# Let's remove empty values
clean_data_dropna = clean_data.dropna(subset=['circonference_cm', 'hauteur_m', 'stade_developpement', 'top_libelle_francais'])

sns.jointplot(data=clean_data_dropna,
    x="circonference_cm", 
    y="hauteur_m", 
    hue="stade_developpement",
    hue_order=['Jeune', 'Jeune Adulte', 'Adulte', 'Mature'],
    height=10,
)


In [ ]:
# Display box plots for trees height and circumference per development stage
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,8))
ax1.set_title("Hauteur par stade de développement")
ax2.set_title("Circonférence par stade de développement")

sns.boxplot(data=clean_data_dropna,
    x="stade_developpement", 
    y="hauteur_m",
    order=['Jeune', 'Jeune Adulte', 'Adulte', 'Mature'],
    ax=ax1,
)

sns.boxplot(data=clean_data_dropna,
    x="stade_developpement", 
    y="circonference_cm",
    order=['Jeune', 'Jeune Adulte', 'Adulte', 'Mature'],
    ax=ax2,
)


Nous voyons qu'il y a des arbres qui ont une taille anormale par rapport à leur stade de développement. Il faudrait contrôler leur santé et leur apporter les soins nécessaires (engrais, arrosage, traitements, ...).



### Où sont situés les arbres qui vont nécessiter le plus d'entretien ?

Plus un arbre est grand, plus il nécessitera de techniciens, de temps, de matériel, d'arrosage et de produits pour son entretien. Maintenant que nous avons éliminé les valeurs aberrantes, nous allons cartographier les arbres en les pondérant avec leur hauteur.


In [ ]:
# Display the trees on a map, weighted by size
fig = px.density_mapbox(clean_data, 
    lat='geo_point_2d_a', lon='geo_point_2d_b',
    z='hauteur_m',
    hover_data=['circonference_cm', 'hauteur_m', 'arrondissement', 'lieu', 'domanialite'],
    radius=2,
    zoom=10,
    mapbox_style="open-street-map",
    title="Localisation des arbres nécessitant le plus de moyens",
    width=1000,
    height=800,
)
fig.show()


### Quels sont les arbres les plus plantés actuellement ?

Nous allons travailler sur les données de catégories d'arbes, en se limitant aux valeurs les plus représentatives. Nous allons chercher à observer quel sont les types d'arbres les plus représentés selon leur type, et leur localisation.


In [ ]:
# Visualize repartition of type of trees
freq = clean_data['top_libelle_francais'].value_counts()
fig = px.pie(freq,
    names=freq.index, 
    values=freq.values,
    title="Types d'arbres",
)
fig.update_traces(
    textposition='inside',
    textinfo='percent+label'
)
fig.show()


Nous voyons ici que seules 4 essences d'abres représentent plus de 50% des arbres plantés. Cette information est importante pour adapter le matériel et les produits nécessaires à l'enretien des arbres.
Cette information montre aussi qu'il pourrait y avoir un problème diversité des essences et donc de résilience du parc arboricole de Paris.


In [ ]:
# Most common trees per burrough
table = clean_data.pivot_table(
    values='id',
    index='arrondissement',
    columns='top_libelle_francais',
    aggfunc='count',
    observed=True,
)
fig = px.imshow(table,
    title="Type d'arbre par arrondissement",
    width=1000,
    height=800,
)
fig.show()


Nous voyons ici que nous avons beaucoup de peupliers, notamment dans le 7ème, le 12ème et le 16ème, ainsi que des marroniers dans le 8ème et le 16ème arrondissement.
Cette information permet de dimensionner et répartire géographiquement les équipes et le matériel en fonction des types d'arbres plantés dans chaque arrondissement.


In [ ]:
# Most common trees per age
table = clean_data.pivot_table(
    values='id',
    index='stade_developpement',
    columns='top_libelle_francais',
    aggfunc='count',
    observed=True,
)
fig = px.imshow(table,
    title="Type d'arbre par stade de développement",
    width=1000,
    height=400,
)
fig.show()


Nous voyons ici que la plupart des arbres sont des platanes adultes.
Cette information permet d'optimiser les achats et le stockage du materiel et des produits adaptés spécifiquement à l'entretien de ces arbres.

---

_[Licence GPL-v3](https://github.com/fleuryc/oc_ingenieur-ia_P2-Participez-a-un-concours-sur-la-Smart-City/blob/main/LICENSE)_
